In [1]:
import config.settings as settings
import utils.system as system

import sqlite3
import pandas as pd
import time
import plotly.express as px


In [21]:

db_name = 'b3 .db'
db_name = 'b3 COMUNICACOES.db'
db_path = 'd:\\Fausto Stangler\\Documentos\\Python\\ASW\\backend\\data\\' + db_name
db_path = 'd:\\Fausto Stangler\\Documentos\\Python\\ASW\\backend\\data\\b3  math.db'
# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Read the finsheet table into a DataFrame
query = "SELECT * FROM finsheet"
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

df['quarter'] = pd.to_datetime(df['quarter'], errors='coerce')
df = df.dropna(subset=['quarter'])
df = df[df['quarter'] > pd.to_datetime('2010-12-31')]

df.sort_values(by=['setor', 'subsetor', 'segmento', 'company_name', 'quarter', 'conta'], inplace=True)
df['valor'] = df['valor']/10**6
df.head(5)

,nsd,tipo,setor,subsetor,segmento,company_name,quadro,quarter,conta,descricao,valor,version
1573,8130,Dados da Empresa,,,,,Composição do Capital,2011-03-31,00.01.01,Ações Ordinárias ON,112.549326,1
1574,7413,Dados da Empresa,,,,,Composição do Capital,2011-03-31,00.01.01,Ações Ordinárias ON,402.202000,2
1575,9499,Dados da Empresa,,,,,Composição do Capital,2011-03-31,00.01.01,Ações Ordinárias ON,0.023000,3
1576,15003,Dados da Empresa,,,,,Composição do Capital,2011-03-31,00.01.01,Ações Ordinárias ON,65.863000,3
1577,19297,Dados da Empresa,,,,,Composição do Capital,2011-03-31,00.01.01,Ações Ordinárias ON,15.000000,3


In [22]:
df['company_name'].unique()

array(['', 'ALTHAIA SA INDUSTRIA FARMACEUTICA',
       'ALUBAR METAIS E CABOS SA',
       'ANHANGUERA EDUCACIONAL PARTICIPACOES SA', 'ASSURUA 5 ENERGIA SA',
       'BRASIL TECNOLOGIA E PARTICIPACOES SA',
       'CANAL COMPANHIA DE SECURITIZACAO', 'CLI SUL SA',
       'CONCESSIONARIA DE RODOVIAS NOROESTE PAULISTA SA',
       'CONCESSIONARIA RODOVIAS DO TRIANGULO SPE SA',
       'CONCESSIONARIA ROTA DE SANTA MARIA SA', 'CREMER SA',
       'EUROFARMA LABORATORIOS SA', 'EVORA SA',
       'FISIA COMERCIO DE PRODUTOS ESPORTIVOS SA',
       'HELIO VALGAS SOLAR PARTICIPACOES SA', 'LG INFORMATICA SA',
       'ORIZON MEIO AMBIENTE SA', 'PARSAN SA', 'PRUMO LOGISTICA SA',
       'SANTANDER LEASING SA ARRENDAMENTO MERCANTIL',
       'TURBI COMPARTILHAMENTO DE VEICULOS SA', 'UNIDAS LOCADORA SA'],
      dtype=object)

In [29]:
df['company_name'].unique()
company = 'ANHANGUERA EDUCACIONAL PARTICIPACOES SA'
conta = '3.01'
tipo = 'DFs Consolidadas'

filter = (
    (df['company_name'] == company) & 
    (df['conta'] == conta) & 
    (df['tipo'] == tipo)
    )
df_filtered = df[filter][['quarter', 'conta', 'valor']]
# Create the Plotly plot
fig = px.line(df_filtered, x='quarter', y='valor', title=f'Quarterly Values for {company} - Conta {conta} (Non-zero)')

# Show the plot
fig.show()

df_filtered


,quarter,conta,valor
93431,2011-03-31,3.01,3055.620
93992,2011-06-30,3.01,267.484
94570,2011-09-30,3.01,309.229
95160,2011-12-31,3.01,-2400.165
95752,2012-03-31,3.01,417.726
96332,2012-06-30,3.01,404.319
96917,2012-09-30,3.01,422.204
97486,2012-12-31,3.01,363.135
98077,2013-03-31,3.01,471.312
98650,2013-06-30,3.01,451.712
